<a href="https://colab.research.google.com/github/Hkachhadiya/Task_1_ML/blob/main/Harsh_Task4_Movie_Recommenders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162728 sha256=40286903efde874d61ff6e69dd51739872c59c4c58dca558ef775028fb0f87e8
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


# **Step-1**

In [5]:
!pip install lightfm
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808329 sha256=0c6a7c9cd0f19350b3ede4ca4255a53dea9c505bc93c47b6c705a953294ce980
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


# **Step-2**

In [9]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')


In [13]:
tags

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078
...,...,...,...,...
465559,138446,55999,dragged,1358983772
465560,138446,55999,Jason Bateman,1358983778
465561,138446,55999,quirky,1358983778
465562,138446,55999,sad,1358983772


In [28]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [29]:
ratings

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
...,...,...,...,...
20000258,138493,68954,4.5,1258126920
20000259,138493,69526,4.5,1259865108
20000260,138493,69644,3.0,1260209457
20000261,138493,70286,5.0,1258126944


# **Step-3**

In [17]:
tags['tag'] = tags['tag'].fillna('')
tags['tag'] = tags['tag'].astype(str)


# **Step-4**

In [18]:
movie_tags = tags.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()

In [19]:
movies_metadata = pd.merge(movies, movie_tags, on='movieId', how='left')

# **Step-5**

In [20]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_metadata['tag'].values.astype('U'))  # Convert to Unicode strings


# **Step-6**

In [22]:
dataset = Dataset()
dataset.fit(users=ratings['userId'], items=movies['movieId'])
(interactions, weights) = dataset.build_interactions([(user, item, rating) for user, item, rating in ratings[['userId', 'movieId', 'rating']].values])
model = LightFM(loss='warp')
model.fit(interactions, epochs=30, num_threads=2)

# **Step-7**

In [24]:
def recommend(user_id, model, interactions, movies_metadata, top_n=10):
    n_users, n_items = interactions.shape
    scores = model.predict(user_id, np.arange(n_items))
    top_items = movies_metadata.iloc[np.argsort(-scores)[:top_n]]
    return top_items

# **Step-8**

In [27]:
user_id = 1
recommended_movies = recommend(user_id, model, interactions, movies_metadata)
print("Recommended Movies for User {}: ".format(user_id))
print(recommended_movies[['movieId', 'title']])

Recommended Movies for User 1: 
      movieId                                              title
257       260          Star Wars: Episode IV - A New Hope (1977)
1184     1210  Star Wars: Episode VI - Return of the Jedi (1983)
583       589                  Terminator 2: Judgment Day (1991)
1171     1196  Star Wars: Episode V - The Empire Strikes Back...
1188     1214                                       Alien (1979)
2486     2571                                 Matrix, The (1999)
476       480                               Jurassic Park (1993)
1175     1200                                      Aliens (1986)
1104     1127                                  Abyss, The (1989)
1212     1240                             Terminator, The (1984)
